### Extract data from JSON

In [1]:
import pandas as pd
import pickle
from time import time
from datetime import datetime, timedelta
import numpy as np
from pprint import pprint
from multiprocessing import Pool, cpu_count
from os import listdir, remove
from os.path import exists
import json
import statistics

In [2]:
def readCSV(file_name):
    df = pd.read_csv(file_name, index_col=0)
    try:
        df['DateTimeTime'] = pd.to_datetime(df['DateTimeTime'])
    except:
        pass
    try:
        df['Date'] = pd.to_datetime(df['Date']).apply(datetime.date)
    except:
        pass
    try:
        df['Time'] = pd.to_datetime(df['Time']).apply(datetime.time)
    except:
        pass

    return df

In [3]:
def reindex(dataframe , sort_columns = ['DateTimeTime' , 'Screen' , 'Movie']):
    dataframe = dataframe.sort_values(by=sort_columns)
    dataframe.index = np.arange(len(dataframe))
    return dataframe

### Forming the basic keys (movie , showtime , screen)

In [4]:
def initializeDataframe(d, norm_df, df_columns):
    temp_df_list = []
    for movie, date_slot in d.items():
        for date_time, data in date_slot.items():
            for screen, values in data.items():
                max_trans_time = values[12]
                row = [movie, date_time, screen, max_trans_time]
                temp_df_list.append(row)
    norm_df = pd.DataFrame(temp_df_list, columns=df_columns)
    norm_df['DateTimeTime'] = pd.to_datetime(norm_df['DateTime'])
    norm_df['DateTime'] = norm_df['DateTime'].apply(np.str)
    norm_df['Date'] = norm_df['DateTimeTime'].apply(datetime.date)
    norm_df['Time'] = norm_df['DateTimeTime'].apply(datetime.time)
    norm_df['MaxTransTime'] = pd.to_datetime(norm_df['MaxTransTime'])
    return reindex(norm_df)

### Get the mean value for given dataframe and column

In [5]:
def getNormalizationParams(dataframe, column):
    try:
        values = dataframe[column].values
        return (statistics.mean(values), statistics.stdev(values))
    except:
        return None

In [6]:
def getMeanDeviation(dataframe):
    norm_columns = norm_df.columns
    norm_dict = {}
    for column in norm_columns:
        ret_mean = getNormalizationParams(norm_df, column)
        if ret_mean != None:
            norm_dict[column] = ret_mean

    return norm_dict

### Removing Overlapping Shows in a day

In [7]:
def showsPerDay(dataframe):
    shows_on_date = {}
    for d in dataframe['Date'].unique():
        n_shows = len(dataframe[dataframe['Date'] == d])
        if n_shows not in shows_on_date:
            shows_on_date[n_shows] = []
        shows_on_date[n_shows].append(d)
    return shows_on_date


def removeOverlappingShows(dataframe):
    overlapping = []
    screens = dataframe['Screen'].unique()
    for screen in screens:
        screen_df = dataframe[dataframe['Screen'] == screen]
        for date in screen_df['Date'].unique():
            temp_df = screen_df[screen_df['Date'] == date]
            for i in range(len(temp_df)-1):
                if temp_df.iloc[i+1]['DateTimeTime'] - temp_df.iloc[i]['DateTimeTime'] <= timedelta(hours=1, minutes=30):
                    overlapping.append(temp_df.index[i+1] if temp_df.iloc[i]['MaxTransTime']
                                       > temp_df.iloc[i+1]['MaxTransTime'] else temp_df.index[i])
    overlapping = list(set(overlapping))
    dataframe.drop(overlapping, inplace=True)
    print(f'Dropped {len(overlapping)} rows')

    return reindex(dataframe)

### New adapter to pool

In [8]:
def getSupply(dataframe):
    supply = []
    for x in range(len(dataframe)):
        supply.append((x, dataframe.iloc[x]))

    return supply

In [9]:
def adapterToPool(dataframe, function, supply=[]):

    if len(supply) == 0:
        supply = getSupply(dataframe)

    hold_values = []

    try:
        pool = Pool(num_pool_to_use)
        hold_values = pool.map(function, supply)
    finally:
        pool.close()
        pool.join()

#     return pd.D(list(hold_values) , axis = 0)
    return pd.DataFrame(hold_values)

### Generic Data Extraction

In [10]:
def genericEnclosingFunction(supply):
    def addCapacity(movie, date_time, screen):
        cap = d[movie][date_time][screen][9]
        return np.int(cap)

    def addHeld(movie, date_time, screen):
        seats_held = d[movie][date_time][screen][8]
        return np.int(max(seats_held, 0))

    def assignSlot(movie, date_time, screen):
        time = pd.to_datetime(date_time)
        hour = time.hour

        if hour >= 20:
            slot = 4  # night show
        elif hour >= 17:
            slot = 3  # evening show
        elif hour >= 14:
            slot = 2  # matinee show
        elif hour >= 11:
            slot = 1  # noon show
        else:
            slot = 0  # morning show
        return slot

    def timeToShow(movie, date_time, screen):
        pred_time = pd.to_datetime(date_time) - timedelta(days=1)
        pred_time = pred_time.replace(
            hour=prediction_hour, minute=prediction_min)
        pred_delta = pd.to_datetime(date_time) - pred_time
        return ((pred_delta.days*24) + (pred_delta.seconds)*1.0/3600)

    def addSeatsAvailAtShow(movie, date_time, screen):
        held = d[movie][date_time][screen][8]
        cap = d[movie][date_time][screen][9]
        final_seats_sold = d[movie][date_time][screen][7] + held
        final_seats_sold = min(final_seats_sold, cap)
        avail_at_show = cap - final_seats_sold
        return max(avail_at_show, 0)

    def addFinalOccPer(occ_at_show, cap):
        return occ_at_show * 100.0 / cap

    # Works on prediction time (sundar)
    def occupAtPredTime(movie, date_time, screen):

        occup = d[movie][date_time][screen][8]  # seats held
        time_stamps = d[movie][date_time][screen][2]
        book_stamps = d[movie][date_time][screen][1]
        show_time = pd.to_datetime(date_time)
        prev_day = show_time - timedelta(days=1)
        prev_day = prev_day.replace(
            hour=prediction_hour, minute=prediction_min)
        ind = -1

        for i in range(len(time_stamps)):
            if pd.to_datetime(time_stamps[i]) == prev_day:
                ind = i
                break

        for i in range(ind + 1):
            occup += book_stamps[i]
        return occup

    def addSeatsSoldPerTrans(movie, date_time, screen):
        seats_sold_per_trans = d[movie][date_time][screen][10]
        return seats_sold_per_trans

    index, row = supply
    movie = row['Movie']
    date_time = row['DateTime']
    screen = row['Screen']

    capacity = addCapacity(movie, date_time, screen)
    held = addHeld(movie, date_time, screen)
    slot = assignSlot(movie, date_time, screen)
    time_to_show = timeToShow(movie, date_time, screen)

    seats_avail_at_show = addSeatsAvailAtShow(movie, date_time, screen)
    seats_occ_at_show = capacity - seats_avail_at_show
    occ_per = seats_occ_at_show/capacity

    occ_at_pred = occupAtPredTime(movie, date_time, screen)
    avail_at_pred = capacity - occ_at_pred

    seats_sold_per_trans = addSeatsSoldPerTrans(movie, date_time, screen)

    row['Capacity'] = capacity
    row['Held'] = (max(held, 0) / capacity)
    row['Slot'] = slot
    row['TimeToShow'] = time_to_show
    row['AvailAtPred'] = (max(avail_at_pred, 0) / capacity)
    row['OccAtPred'] = min(occ_at_pred, capacity)
    row['AvailAtShow'] = max(seats_avail_at_show, 0)
    row['OccAtShow'] = min(seats_occ_at_show, capacity)
    row['OccPer'] = row['OccAtShow']/capacity
    row['SeatsSoldPerTrans'] = seats_sold_per_trans

    return row

### The Reverse Percentage Factor

In [11]:
def percentageEnclosingFunction(supply):
    #Percentage (sundar)
    def percentage(supply):
        per = np.float(set_percentage)
        index, row = supply
        film = row['Movie']
        showtime = row['DateTime']
        screen = row["Screen"]
        capacity = d[film][showtime][screen][9]
        held = d[film][showtime][screen][8]
        time_stamps = d[film][showtime][screen][2]
        book_stamps = list(d[film][showtime][screen][1])
        sum_stamps = [0]
        show_time = row['DateTimeTime']
        prev_day = show_time - timedelta(days=1)
        prev_day = prev_day.replace(
            hour=prediction_hour, minute=prediction_min)
        ind = 0

        for i in range(len(book_stamps) - 1):
            sum_stamps.append(book_stamps[i] + sum_stamps[-1])

        for i in range(len(time_stamps)):
            if pd.to_datetime(time_stamps[i]) == prev_day:
                ind = i + 1
                break

        for i in range(ind):
            if((held + sum_stamps[i]*1.0)/capacity >= per):
                cond_stamp = pd.to_datetime(time_stamps[i])
                time_diff = pd.to_datetime(time_stamps[-1]) - cond_stamp
                return row['Capacity']/((24*time_diff.days) + (time_diff.seconds/3600))

        return -1

    index, row = supply
    per = percentage(supply)
    row[str(set_percentage * 100)] = per
    return row

### Holiday Extraction. Single Threaded and Fast !

In [12]:
# Single Threaded and Faster!
def holidayEncoding(norm_df):

    norm_df['ShowDayHoliday'] = 0
    norm_df['NextDayHoliday'] = 0

    # Dictionary Containing the number of shows per day
    shows_per_day = norm_df[['Date', 'Movie']].groupby(
        by='Date').count().to_dict()['Movie']

    holidays = np.array([date for date, nshows in shows_per_day.items(
    ) if nshows >= holiday_num_show_factor])
    # Updating the Show day holidays to 1
    norm_df['ShowDayHoliday'][norm_df['Date'].isin(holidays)] = 1
    # Updating the Next day holiday to 1
    holidays = holidays - timedelta(days=1)
    norm_df['NextDayHoliday'][norm_df['Date'].isin(holidays)] = 1

    return norm_df

### Calculating days since release

In [13]:
def daysSinceRelease(supply):
    index, row = supply
    movie = row['Movie']
    date = row['Date']
    release_date = pd.to_datetime(release_dates[movie])
    day_diff = date - release_date.date()
    row['DaysSinceRelease'] = (int)(
        min(day_diff.days, cap_max_days_since_release))
    return row

In [14]:
def calculateDSR(norm_df):
    movies = norm_df['Movie'].unique()
    global release_dates
    movie_details = pd.DataFrame()
    release_dates = {}
    try:
        movie_details = readCSV('movie_details.csv')
    except:
        pass
    for movie in movies:
        try:
            release_dates[movie] = pd.to_datetime(
                movie_details['ReleaseDate'][movie_details['Movie'] == movie].value[0])
        except:
            release_dates[movie] = min(
                norm_df['DateTimeTime'][norm_df['Movie'] == movie])
            next_index = 0
            try:
                next_index = max(movie_details.index) + 1
            except:
                pass
            movie_details = movie_details.set_value(next_index, 'Movie', movie)
            movie_details = movie_details.set_value(
                next_index, 'ReleaseDate', release_dates[movie])

    movie_details.to_csv('movie_details.csv')
    adapterToPool(norm_df, daysSinceRelease)
    del release_dates
    return norm_df

### Adding Show History

In [15]:
def getShowHistory(supply, num_history_points=7):

    index, row = supply
    min_date = row['Date'] - timedelta(days=2)
    history = norm_df[norm_df['Movie'] == row['Movie']][(
        norm_df['Date'] <= min_date)].sort_values(by='Date', ascending=False)

    history_pers = history['OccPer'].values[:num_history_points]
    for i in range(num_history_points):
        try:
            row[f"ShowHistory{i}"] = history_pers[i]
        except IndexError:
            row[f"ShowHistory{i}"] = -1.0

    try:
        row['History_Mean'] = statistics.mean(history_pers)
        row['History_Max'] = max(history_pers)
        row['History_Min'] = min(history_pers)
    except:
        row['History_Mean'] = -1
        row['History_Max'] = -1
        row['History_Min'] = -1
    return row

### Adding Slot History

In [16]:
def getSlotHistory(supply):
        index, row = supply
        date, screen, slot, is_holiday = row['Date']-timedelta(
            days=2), row['Screen'], row['Slot'], row['ShowDayHoliday']
        # Extra days to account for anomalies, hols and nothols
        history = norm_df[(norm_df['Date'] <= date) & (
            norm_df['Date'] > date-timedelta(days=180))]
        history = history[history['Slot'] == slot]

    #     Average Slot History
        avg_slot_history = history[history['ShowDayHoliday'] == is_holiday][['OccPer', 'Date']].groupby(
            'Date').mean().sort_index(ascending=False)['OccPer'].values[0:7].mean()
        history = history[(history['Screen'] == screen) & (
            history['ShowDayHoliday'] == is_holiday)].sort_values(by='Date', ascending=False)
        history = list(history['OccPer'].values[0:7])

        if len(history) != 7:
            history += [-1]*(7-len(history))

        for count in range(len(history)):
            row[f'HolidayBasedHistory{count}'] = history[count]
        row['AverageSlotHistory'] = avg_slot_history
        return row

### Writing the df

In [20]:
def getFeatureDataframe(inp, out=None, dataframe_from='1-Jan-2013'):
    global d, prediction_hour, prediction_min, num_pool_to_use, holiday_num_show_factor, set_percentage, norm_df, cap_max_days_since_release

    if(type(inp) == str):
        d = json.load(open(inp))
    else:
        d = inp

    # time of prediction on previous day, defaults to 3:00pm
    prediction_hour, prediction_min = 15, 0
    # number of processors to use to complete data extraction
    num_pool_to_use = cpu_count() - 1
    # if no. of shows greater than this val , then it is declared holiday else not holiday
    holiday_num_show_factor = 40
    # Hours taken from showtime to reach (set_percentage) fraction of the occupancy
    set_percentage = 1.0
    cap_max_days_since_release = 200
    # Json file containing the data

    df_columns = ['Movie', 'DateTime', 'Screen', 'MaxTransTime']
    count = 0
    norm_df = pd.DataFrame(columns=df_columns)
    norm_df = initializeDataframe(d, norm_df, df_columns)
    norm_df = norm_df[norm_df['Date'] >= pd.to_datetime(dataframe_from).date()]
    print("Created basic DF")

    norm_df['MaxTransTime'] = pd.to_datetime(norm_df['MaxTransTime'])
    norm_df = removeOverlappingShows(norm_df)
    del norm_df['MaxTransTime']
    print("Removed overlapping shows")

    norm_df = adapterToPool(norm_df, genericEnclosingFunction)
    print("Added generic columns")

    set_percentage = 0.025
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.05
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.075
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.10
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.15
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.20
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    print("Added percentage factors (3/6)")

    set_percentage = 0.30
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.40
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    set_percentage = 0.50
    norm_df = adapterToPool(norm_df, percentageEnclosingFunction)
    print("Added percentage factors (6/6)")

    norm_df = holidayEncoding(norm_df)
    print("Added holiday factors")

    norm_df = calculateDSR(norm_df)
    print("Added days since release factor")

    norm_df = adapterToPool(norm_df, getShowHistory)
    print("Added history features")

    norm_df = reindex(norm_df)

    if out != None:
        if '.csv' not in out:
            out += '.csv'
        norm_df.to_csv(out)
        print("Written to csv")

    return norm_df

In [22]:
if __name__ == '__main__':
    getFeatureDataframe(
        '../SPI_movies/features/features_escape_added_minmaxtranstime.json', 'new_final_features.csv')

Created basic DF
Dropped 778 rows
Removed overlapping shows
Added generic columns
Added percentage factors (3/6)
Added percentage factors (6/6)


/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Added holiday factors
Added days since release factor


/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.

Added history features
Written to csv
